In [6]:
import json
with open('intent_texts.json', 'r', encoding='utf-8') as f:
    intent_texts = json.load(f)

In [7]:
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch

num_labels = 2
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

train_encodings = tokenizer(intent_texts['texts'], truncation=True, padding=True)

class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = Dataset(train_encodings, intent_texts['classes'])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

model.save_pretrained("./my_intent_model")
tokenizer.save_pretrained("./my_tokenizer_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/30 [00:00<?, ?it/s]

{'loss': 0.525, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.67}
{'loss': 0.1994, 'learning_rate': 1.6666666666666667e-05, 'epoch': 3.33}
{'loss': 0.0836, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 22.7279, 'train_samples_per_second': 9.24, 'train_steps_per_second': 1.32, 'train_loss': 0.2693375706672668, 'epoch': 5.0}


('./my_tokenizer_model/tokenizer_config.json',
 './my_tokenizer_model/special_tokens_map.json',
 './my_tokenizer_model/vocab.txt',
 './my_tokenizer_model/added_tokens.json',
 './my_tokenizer_model/tokenizer.json')

In [11]:
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs).item()
    labels_map = {0: "general", 1: "platform"}
    return labels_map[pred_label]

print(predict_intent("hello"))
print(predict_intent("hello, can you help me?"))
print(predict_intent("i need your help"))
print(predict_intent("how can i track my order"))
print(predict_intent("can i pay cash for my order"))
print(predict_intent("the order has not arrived yet. i dont want it anymore, i want my money back"))
print(predict_intent("tell me about nespresso"))

general
general
general
platform
platform
platform
platform
